In [24]:
using LinearAlgebra
using SparseArrays
using Arpack

## Building up Hamiltonian without using k space
- Building Hamiltonian for the simplest square lattice heisenberg model
- Using Int64 to represent states
- start by a 2 by 2 lattice, so takes 4 bits out of the 64 bits of an Int64.

Hamiltonian is $H=J\sum_{i=1}^{N}(S_i^z S_{i+1}^z + 1/2(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)) $

First find all neighbours of a given site in the binary representation. then build up a matrix of 16 by 16
The k-th bit of a binary number correspond to the (i,j) site on the lattice, for a lattice of linear size L. Which could be calculated by this following function:

In [25]:
function coordinate(n;L::Int64=2)
    num_sites = L^2
    i::Int64 = Int(ceil(n/L))
    j::Int64 = mod1(n,L)  #site i is at i-th row, j-th column
    return (i,j)
end

function bit_pos(coordinate::Tuple{Int64,Int64};L::Int64=2)
    n = (coordinate[1]-1)*L + coordinate[2]
    return n
end

bit_pos (generic function with 1 method)

# Now Let's Construct Hamiltonian for our problem, by changing neighbor lists

In [26]:
function neib(n::Int64;L::Int64=2)
    coord = coordinate(n,L=L)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L)))    
    if iseven(coord[1]+coord[2])
        push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]-1,L)))
        push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]+1,L)))
    else
        push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]+1,L)))
        push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]-1,L)))
    end
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib,L=L))
    end
    return neibs_bit_pos
end

function neib_list_gen(;L::Int64=L)
    neib_list = Set{Int64}[]
    for n in 1:L^2
        push!(neib_list, neib(n, L=L))
    end
    return neib_list
end

neib_list_gen (generic function with 1 method)

# Now we can construct Hamlitonian for our system
## Let's make use of the Parity Symmetry, starting by constructing Parity States

Our Hamiltonian is originally $H=J\sum_{i=1}^{N}\sum_{\sigma}(S_i^z S_{i+\sigma}^z + h S_i^x) $

We can do a little rotation of our system and construct Hamiltonian in that "direction". The Hamiltonian should be:
$H=J\sum_{i=1}^{N}\sum_{\sigma}(J S_i^x S_{i+\sigma}^x - h S_i^z)$

Equivalently, we can write our Hamiltonian as $H=J\sum_{i=1}^{N}\sum_{\sigma}(\frac{J}{4}(S_i^+ + S_i^-)(S_{i+\sigma}^+ + S_{i+\sigma}^-) - h S_i^z)$

Now we have a nice Parity symmetry comes in handy. The Operator is $\prod_i \sigma_i^z$. The $\sigma_i^z$ basis is exactly the eigenbasis of the parity operator. The parity operator commutes with H, thus divides the Hamiltonian into 2 block diagonal parts. With one part having even total spin, one part having odd total spin. 

Physically we can see this by making use of the fact that $S_i^x S_{i+\sigma}^x$ flips 2 spins at a time. So different sectors can't mix.

First we need to loop over all states, and construct a list of even and odd states.

In [27]:
function chk_parity(state::Int64)
    state_binary = digits!(zeros(Int64, 64), state, base = 2)
    if iseven(sum(state_binary))
        return :even
    else
        return :odd
    end
end

function parity_states_list(;L::Int64=L)
    even_state = Int64[]
    odd_state = Int64[]
    even_state_num = Dict{Int64, Int64}()
    odd_state_num = Dict{Int64, Int64}()
    even_state_tot = 0
    odd_state_tot = 0
    for state in 0:(2^(L^2)-1)
        if chk_parity(state) == :even
            even_state_tot += 1
            push!(even_state, state)
            even_state_num[state] = even_state_tot
        else
            odd_state_tot += 1
            push!(odd_state, state)
            odd_state_num[state] = odd_state_tot
        end
    end
    return Dict{Symbol, Any}(:even_state => even_state, :odd_state => odd_state, :even_state_num => even_state_num, :odd_state_num => odd_state_num, :even_state_tot => even_state_tot, :odd_state_tot => odd_state_tot)
end

parity_states_list (generic function with 1 method)

## Now we could build up function to construct Hamiltonian for even and odd sectors

In [28]:
function Hamiltonian(;L::Int64=2, J=1, h=1, neib_list, state_list, state_num, state_tot)
    H = spzeros(state_tot,state_tot)
    for state in state_list #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites i in a given state
            #diagonal terms interact with h
            if state_binary[i] == 1
                H[state_num[state],state_num[state]] += h/2
            else
                H[state_num[state],state_num[state]] -= h/2
            end   
            # off diagonal terms come from flipping bonds, j is neighbor of i
            for j in neib_list[i] 
                fliped_state = state ⊻ (1<<(i-1))
                fliped_state = fliped_state ⊻ (1<<(j-1))
                H[state_num[state],state_num[fliped_state]] = J/4
            end
        end
    end
    return H
end

Hamiltonian (generic function with 1 method)

## Now, a driver to organize all the functions, and produce Hamiltonian for even and odd sectors

In [29]:
function H_driver(;L::Int64=L, J::Float64=J, h::Float64=h)
    neib_list = neib_list_gen(L=L)
    state_gen = parity_states_list(L=L)
    even_state = state_gen[:even_state]
    even_state_num = state_gen[:even_state_num]
    odd_state_num  = state_gen[:odd_state_num]
    odd_state = state_gen[:odd_state]
    even_state_tot = state_gen[:even_state_tot]
    odd_state_tot = state_gen[:odd_state_tot]
    #now constructine Hamiltonian of the two sectors
    Hamiltonian_even = Hamiltonian(;L=L, J=J, h=h, neib_list=neib_list, state_list=even_state, state_num=even_state_num, state_tot=even_state_tot)
    Hamiltonian_odd = Hamiltonian(;L=L, J=J, h=h, neib_list=neib_list, state_list=odd_state, state_num=odd_state_num, state_tot=odd_state_tot)
    return Dict(:even => Hamiltonian_even, :odd => Hamiltonian_odd)
end

H_driver (generic function with 1 method)

# LET'S RUN IT !!!

In [30]:
L=2;J=1.0;h=1.0
H_blocks = H_driver(L=L, J=J, h=h)
println(H_blocks[:even])
println(H_blocks[:odd])


  [1, 1]  =  -2.0
  [2, 1]  =  0.25
  [3, 1]  =  0.25
  [4, 1]  =  0.25
  [5, 1]  =  0.25
  [6, 1]  =  0.25
  [7, 1]  =  0.25
  [1, 2]  =  0.25
  [2, 2]  =  0.0
  [3, 2]  =  0.25
  [4, 2]  =  0.25
  [5, 2]  =  0.25
  [6, 2]  =  0.25
  [8, 2]  =  0.25
  [1, 3]  =  0.25
  [2, 3]  =  0.25
  [3, 3]  =  0.0
  [4, 3]  =  0.25
  [5, 3]  =  0.25
  [7, 3]  =  0.25
  [8, 3]  =  0.25
  [1, 4]  =  0.25
  [2, 4]  =  0.25
  [3, 4]  =  0.25
  [4, 4]  =  0.0
  [6, 4]  =  0.25
  [7, 4]  =  0.25
  [8, 4]  =  0.25
  [1, 5]  =  0.25
  [2, 5]  =  0.25
  [3, 5]  =  0.25
  [5, 5]  =  0.0
  [6, 5]  =  0.25
  [7, 5]  =  0.25
  [8, 5]  =  0.25
  [1, 6]  =  0.25
  [2, 6]  =  0.25
  [4, 6]  =  0.25
  [5, 6]  =  0.25
  [6, 6]  =  0.0
  [7, 6]  =  0.25
  [8, 6]  =  0.25
  [1, 7]  =  0.25
  [3, 7]  =  0.25
  [4, 7]  =  0.25
  [5, 7]  =  0.25
  [6, 7]  =  0.25
  [7, 7]  =  0.0
  [8, 7]  =  0.25
  [2, 8]  =  0.25
  [3, 8]  =  0.25
  [4, 8]  =  0.25
  [5, 8]  =  0.25
  [6, 8]  =  0.25
  [7, 8]  =  0.25
  [8, 8]  =  2.